In [ ]:
%load_ext autoreload
%autoreload 2
import math
from pathlib import Path

import h5py as h5
import numpy as np
from scipy import constants, signal, stats, optimize
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt

import physics
import plots
import analysis
from definitions import (
    RESULTS_FOLDER,
    MPLSTYLE_FILE,
    SpeciesInfo,
    RunInfo,
    Species
)

run_info = RunInfo(
    electron=SpeciesInfo(
        number_density=12.0e6,
        temperature=100.0,
        charge=-1,
        mass=1.0,
        bulk_velocity=0.0
    ),
    proton=SpeciesInfo(
        number_density=10.0e6,
        temperature=3.0,
        charge=+1,
        mass=1836.152674,
        bulk_velocity=0.0
    ),
    alpha=SpeciesInfo(
        number_density=1.0e6,
        temperature=12.0,
        charge=+2,
        mass=7294.29953,
        bulk_velocity=1.0e5
    )
)

In [ ]:
plots.electricFieldOverSpaceAndTime(
    folder=RESULTS_FOLDER / "proton-alpha-instability-1D",
    info=run_info
)
plt.show()

In [ ]:
plots.averageTemperature3DOverTime(
    folder=RESULTS_FOLDER / "proton-alpha-instability-1D",
    info=run_info
)
plt.show()

In [ ]:
for s in Species:
    plots.velocityDistributionOverTime(
        folder=RESULTS_FOLDER / "proton-alpha-instability-1D",
        info=run_info,
        species=s
    )
plt.show()

In [ ]:
plots.electricFieldEnergyOverTime(
    folder=RESULTS_FOLDER / "proton-alpha-instability-1D",
    info=run_info
)
plt.show()

# Particle variation

In [ ]:
variation_folder = RESULTS_FOLDER / "particle_variation"
for particle_folder in sorted(variation_folder.glob("particles_*")):
    plots.multiElectricFieldEnergyOverTime(
        particle_folder, run_info,
        identifer=f"particles_per_cell_{int(particle_folder.stem[-4:])}"
    )

In [ ]:
plots.particleVariationGrowthRate(run_info)
plt.show()

In [ ]:
for species in Species:
    plots.particleVariationTemperature3D(species)
plt.show()

In [ ]:
plots.particleVariationEnergyVsTime(info=run_info)
plt.show()

In [ ]:
for species in Species:
    plots.particleVariationTemperatureXDiff(run_info, species)
plt.show()

In [ ]:
for species in Species:
    plots.particleVariationTemperatureXVsTime(run_info, species)
plt.show()

### Wave number

In [ ]:
plots.particleVariationWavenumber(info=run_info)
plt.show()

In [ ]:
plots.particleVariationFrequency(info=run_info)
plt.show()